In [1]:
import os
os.chdir("../")
from joblib import Parallel, delayed

import semiolog as slg

semiotic = slg.Cenematic("fr_wiki",requested_cpu=4)

Vocabulary will not be loaded from file.



In [31]:
from collections import Counter
import operator
from functools import reduce
import regex as re
from tqdm.notebook import trange

In [3]:
%load_ext line_profiler

In [4]:
def count(chain):
    count = Counter(zip(chain,chain[1:]))
    return count

def normalize(chain):
    return list((" ".join(chain)).replace(" ",""))

def agglutinate_chain(pair, chain_list):
    chain_list = " ".join(chain_list) 
    bigram = re.escape(" ".join(pair))
    p = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")
    chain_list = p.sub("".join(pair), chain_list)
    chain_list = chain_list.split()
    return chain_list

In [30]:
import time
bla = time.time()
time.sleep(1)
time.time() - bla

1.0020711421966553

In [32]:


chunksize = int(semiotic.corpus.train_len/semiotic.config.system.cpu_count)+1
chains = [semiotic.corpus.train[i*chunksize:i*chunksize+chunksize] for i in range(semiotic.config.system.cpu_count)]

# from multiprocessing import Manager

# manager = Manager()


def my_func(chains = chains):
    with Parallel(n_jobs=semiotic.config.system.cpu_count, prefer="processes") as parallel:
        chains = [normalize(chain) for chain in chains]
        # chains = manager.list(chains)
        n_iter = 0
        for i in trange(10):
        # while n_iter<10:
            result = parallel([delayed(count)(chain) for chain in chains])
            pairs = reduce(operator.add, result)
            best_pair = "".join(pairs.most_common(1)[0][0])
            chains = parallel([delayed(agglutinate_chain)(best_pair,chain) for chain in chains])
            # chains = [agglutinate_chain(best_pair, chain) for chain in chains]
            # n_iter += 1
        freqs = parallel([delayed(Counter)(chain) for chain in chains])
        freq = reduce(operator.add, freqs)
    return "Done"

In [33]:
%lprun -f my_func my_func()

Timer unit: 1e-06 s

Total time: 33.4913 s
File: <ipython-input-32-949c3d27e8d2>
Function: my_func at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def my_func(chains = chains):
    10         1        870.0    870.0      0.0      with Parallel(n_jobs=semiotic.config.system.cpu_count, prefer="processes") as parallel:
    11         1     353064.0 353064.0      1.1          chains = [normalize(chain) for chain in chains]
    12                                                   # chains = manager.list(chains)
    13         1          2.0      2.0      0.0          n_iter = 0
    14        11      73128.0   6648.0      0.2          for i in trange(10):
    15                                                   # while n_iter<10:
    16        10   16079453.0 1607945.3     48.0              result = parallel([delayed(count)(chain) for chain in chains])
    17        10     604748.0  60474.8      1.8              pairs

In [34]:

chain = semiotic.corpus.train

def my_seq(chain=chain):
    chain = normalize(chain)
    n_iter = 0
    for i in trange(10):
        pairs = count(chain)
        best_pair = "".join(pairs.most_common(1)[0][0])
        chain = agglutinate_chain(best_pair,chain)
    freq_seq = Counter(chain)
    return "Done"

In [35]:
%lprun -f my_seq my_seq()

Timer unit: 1e-06 s

Total time: 34.4098 s
File: <ipython-input-34-7e810f7d9495>
Function: my_seq at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def my_seq(chain=chain):
     4         1     408943.0 408943.0      1.2      chain = normalize(chain)
     5         1          1.0      1.0      0.0      n_iter = 0
     6        11     308567.0  28051.5      0.9      for i in trange(10):
     7        10   22352645.0 2235264.5     65.0          pairs = count(chain)
     8        10      10141.0   1014.1      0.0          best_pair = "".join(pairs.most_common(1)[0][0])
     9        10   10541100.0 1054110.0     30.6          chain = agglutinate_chain(best_pair,chain)
    10         1     788428.0 788428.0      2.3      freq_seq = Counter(chain)
    11         1          2.0      2.0      0.0      return "Done"

In [9]:
for l,r in list(zip(freq.most_common(),freq_seq.most_common())):
    if l!=r:
        print(l,r)

In [10]:
n = 50
list(zip(freq.most_common(n),freq_seq.most_common(n)))

[(('a', 441290), ('a', 441290)),
 (('t', 393100), ('t', 393100)),
 (('i', 368352), ('i', 368352)),
 (('e', 363301), ('e', 363301)),
 (('u', 360618), ('u', 360618)),
 (('r', 335156), ('r', 335156)),
 (('l', 300385), ('l', 300385)),
 (('o', 267331), ('o', 267331)),
 (('s', 263724), ('s', 263724)),
 (('c', 211612), ('c', 211612)),
 (('d', 181825), ('d', 181825)),
 (('n', 180368), ('n', 180368)),
 (('es', 176590), ('es', 176590)),
 (('p', 176275), ('p', 176275)),
 (('m', 173766), ('m', 173766)),
 (('é', 162941), ('é', 162941)),
 (('en', 122018), ('en', 122018)),
 (('on', 112878), ('on', 112878)),
 (('de', 103861), ('de', 103861)),
 ((',', 93686), (',', 93686)),
 (('an', 91953), ('an', 91953)),
 (('g', 80184), ('g', 80184)),
 (('v', 79655), ('v', 79655)),
 (('le', 76879), ('le', 76879)),
 (('re', 72665), ('re', 72665)),
 (('ti', 70646), ('ti', 70646)),
 (('h', 69910), ('h', 69910)),
 (('er', 65824), ('er', 65824)),
 (('f', 65681), ('f', 65681)),
 (('.', 64470), ('.', 64470)),
 (('is', 63108

In [36]:
def parallel_chain(chain, n_of_parts, overlap = 0):
    """
    Breaks the chain in n chunks to compute best pair of terms. "overlap" gives the number of terms of overlap between chunks to alow to ensure that no pair of terms is lost due to the break.
    """
    chunk_size = int(len(chain) / n_of_parts)+1            
    for i in range(0, len(chain), chunk_size):
        yield chain[i : i + chunk_size + overlap]

In [38]:
list(parallel_chain(list("azelazlkejalkzejakzle"),4))

[['a', 'z', 'e', 'l', 'a', 'z'],
 ['l', 'k', 'e', 'j', 'a', 'l'],
 ['k', 'z', 'e', 'j', 'a', 'k'],
 ['z', 'l', 'e']]